In [ ]:
import {
  REGISTRAR_USERNAME,
  REGISTRAR_PASSWORD,
} from '../v1-to-v2-data-migration/helpers/vars.ts'
import { authenticate } from '../v1-to-v2-data-migration/helpers/authentication.ts'

const token = await authenticate(REGISTRAR_USERNAME, REGISTRAR_PASSWORD)
token


In [ ]:
import { fetchEvents } from '../v1-to-v2-data-migration/helpers/formsHandlers.ts'
import { readCsvHeader } from './helpers/csvHelpers.ts'

const pathToBirthCsv = './sourceData/Birth_Register.csv'
const pathToDeathCsv = './sourceData/Death_Register.csv'
const pathToMarriageCsv = './sourceData/Marriage_Register.csv'
const pathToAdoptionCsv = './sourceData/Adoption_Register.csv'
const pathToDeedpollCsv = './sourceData/Deedpoll.csv'

const csvFields = {
  birth: await readCsvHeader(pathToBirthCsv),
  death: await readCsvHeader(pathToDeathCsv),
  marriage: await readCsvHeader(pathToMarriageCsv),
  adoption: await readCsvHeader(pathToAdoptionCsv),
  deedpoll: await readCsvHeader(pathToDeedpollCsv),
}
const events = await fetchEvents(token)


In [ ]:
import { extractFieldType } from '../v1-to-v2-data-migration/helpers/utils.ts'
import { csvToJson } from './helpers/csvHelpers.ts'

const ignoredFields = [
  'DIVIDER',
  'PARAGRAPH',
  'BULLET_LIST',
  'HEADING3',
  'SIGNATURE',
  'collector.*',
  'requester.*',
  'review.*',
  'fees.*',
  'lateFee.*',
  'documents.*',
]

const eventFields = events.reduce(
  (allEvents, event) => {
    allEvents[event.id] = [
      ...new Set(
        extractFieldType(event, 'fields')
          .filter((x) => !ignoredFields.includes(x.type))
          .map((f) => f.id)
          .filter((x) => x)
          .filter(
            (x) =>
              !ignoredFields.some((pattern) => new RegExp(pattern).test(x)),
          ),
      ),
    ]
    return allEvents
  },
  {} as Record<string, string[]>,
)

Deno.writeFileSync(
  './formData/v1FormFields.json',
  new TextEncoder().encode(JSON.stringify(csvFields, null, 2)),
)
Deno.writeFileSync(
  './formData/v2EventFields.json',
  new TextEncoder().encode(JSON.stringify(eventFields, null, 2)),
)

const exampleBirth = await csvToJson(pathToBirthCsv, 2)
Deno.writeFileSync(
  './sourceData/exampleBirthRecord.json',
  new TextEncoder().encode(JSON.stringify(exampleBirth, null, 2)),
)

const exampleDeath = await csvToJson(pathToDeathCsv, 2)
Deno.writeFileSync(
  './sourceData/exampleDeathRecord.json',
  new TextEncoder().encode(JSON.stringify(exampleDeath, null, 2)),
)

const exampleMarriage = await csvToJson(pathToMarriageCsv, 2)
Deno.writeFileSync(
  './sourceData/exampleMarriageRecord.json',
  new TextEncoder().encode(JSON.stringify(exampleMarriage, null, 2)),
)

const exampleAdoption = await csvToJson(pathToAdoptionCsv, 2)
Deno.writeFileSync(
  './sourceData/exampleAdoptionRecord.json',
  new TextEncoder().encode(JSON.stringify(exampleAdoption, null, 2)),
)

const exampleDeedpoll = await csvToJson(pathToDeedpollCsv, 2)
Deno.writeFileSync(
  './sourceData/exampleDeedpollRecord.json',
  new TextEncoder().encode(JSON.stringify(exampleDeedpoll, null, 2)),
)


In [ ]:
import birthMappings  from './mappings/birthEventMappings.ts'
import deathMappings from './mappings/deathEventMappings.ts'
import marriageMappings from './mappings/marriageEventMappings.ts'
import marriageLicenceMappings from './mappings/marriageLicenceEventMappings.ts'
import adoptionMappings from './mappings/adoptionEventMappings.ts'
import nameChangeMappings from './mappings/nameChangeEventMappings.ts'

// Show unmapped CSV fields
// Show unmapped event fields
// Show fields with no data

const flatCsvFields = Object.entries(csvFields).flatMap(([eventType, fields]) =>
  fields.map((field) => `${eventType}.${field}`),
)

const unusedCsvFields = flatCsvFields.filter(
  (field) =>
    !Object.values(birthMappings).includes(field) &&
    !Object.values(deathMappings).includes(field) &&
    !Object.values(marriageMappings).includes(field) &&
    !Object.values(marriageLicenceMappings).includes(field) &&
    !Object.values(adoptionMappings).includes(field) &&
    !Object.values(nameChangeMappings).includes(field),
)

unusedCsvFields

const getEmptyFields = async (pathToCsv: string) => {
  const records = await csvToJson(pathToCsv)
  const header = await readCsvHeader(pathToCsv)
  const emptyFields: string[] = []

  header.forEach((field) => {
    const allEmpty = records.every((record) => {
      const value = record[field]
      return value === null || value === undefined || value === ''
    })
    if (allEmpty) {
      emptyFields.push(field)
    }
  })

  return emptyFields
}

const empty = {
  birth: await getEmptyFields(pathToBirthCsv),
  death: await getEmptyFields(pathToDeathCsv),
  marriage: await getEmptyFields(pathToMarriageCsv),
  adoption: await getEmptyFields(pathToAdoptionCsv),
  deedpoll: await getEmptyFields(pathToDeedpollCsv),
}
empty
unusedCsvFields


In [ ]:
import {
  DEFAULT_FIELD_MAPPINGS,
  CUSTOM_FIELD_MAPPINGS,
  AGE_MAPPINGS,
} from './helpers/defaultMappings.ts'
import { ADDRESS_MAPPINGS } from './countryData/addressMappings.ts'
import { NAME_MAPPINGS } from './countryData/nameMappings.ts'
import { COUNTRY_FIELD_MAPPINGS } from './countryData/countryMappings.ts'
import defaultResolver, {
  defaultBirthResolver,
  defaultDeathResolver,
} from './helpers/defaultResolvers.ts'
import { countryResolver } from './countryData/countryResolvers.ts'
import historyResolver from './helpers/historyResolver.ts'
import { v1IgnoreList } from './countryData/ignoreList.ts'

const birthResolver = {
  ...defaultBirthResolver,
  ...defaultResolver,
  ...countryResolver,
}
const deathResolver = {
  ...defaultDeathResolver,
  ...defaultResolver,
  ...countryResolver,
}

const addressMap = Object.fromEntries(
  Object.entries(ADDRESS_MAPPINGS).map(([f, value]) => [
    f,
    Object.keys(value(null))[0],
  ])
)

const nameMap = Object.fromEntries(
  Object.entries(NAME_MAPPINGS).map(([f, value]) => [
    f,
    Object.keys(value(null))[0],
  ])
)

const ageMap = Object.fromEntries(
  Object.entries(AGE_MAPPINGS).map(([f, value]) => [
    f,
    Object.keys(value(''))[0],
  ])
)

const allMappings = {
  ...DEFAULT_FIELD_MAPPINGS,
  ...CUSTOM_FIELD_MAPPINGS,
  ...ageMap,
  ...addressMap,
  ...nameMap,
  ...COUNTRY_FIELD_MAPPINGS,
}

function partition<T>(list: T[], predicate: (item: T) => boolean) {
  const map: T = {}
  const unmapped: T[] = []

  for (const i of list) {
    if (predicate(i)) {
      map[i] = predicate(i)
    } else {
      unmapped.push(i)
    }
  }

  return { map, unmapped }
}

const birthFormMappings = partition(
  birthFormFields.filter((f) => !v1IgnoreList.includes(f)),
  (f) => allMappings[f]
)
const deathFormMappings = partition(
  deathFormFields.filter((f) => !v1IgnoreList.includes(f)),
  (f) => allMappings[f]
)

const unMappedBirthEventFields = birthEventFields.filter(
  (f) =>
    !Object.values(birthFormMappings.map).includes(f) &&
    !Object.keys(historyResolver).includes(f)
)
const unMappedDeathEventFields = deathEventFields.filter(
  (f) =>
    !Object.values(deathFormMappings.map).includes(f) &&
    !Object.keys(historyResolver).includes(f)
)

const badBirthMappings = Object.entries(birthFormMappings.map).filter(
  ([_k, v]) => !birthEventFields.includes(v)
)

const badDeathMappings = Object.entries(deathFormMappings.map).filter(
  ([_k, v]) => !deathEventFields.includes(v)
)

const unmappedFields = {
  birth: {
    missingV1Mappings: birthFormMappings.unmapped,
    extraV2Mappings: unMappedBirthEventFields,
  },
  death: {
    missingV1Mappings: deathFormMappings.unmapped,
    extraV2Mappings: unMappedDeathEventFields,
  },
}

const unResolvedBirthFields = Object.entries(birthFormMappings.map)
  .filter(([_k, v]) => !badBirthMappings.some(([_k1, v1]) => v1 === v))
  .filter(([_k, v]) => !birthResolver[v])

const unResolvedDeathFields = Object.entries(deathFormMappings.map)
  .filter(([_k, v]) => !badDeathMappings.some(([_k1, v1]) => v1 === v))
  .filter(([_k, v]) => !deathResolver[v])


In [ ]:
const render = (list) => {
  if (list.length) {
    console.table(list)
  } else {
    console.log('None \x1b[32m✔\x1b[0m')
  }
}

console.log('Unmapped V1 form fields:')
console.log()

console.log('Birth:')
render(unmappedFields.birth.missingV1Mappings)

console.log('Death:')
render(unmappedFields.death.missingV1Mappings)
console.log()
console.log('V1 form fields mapped to a V2 field that does not exist:')
console.log('Birth:')
render(badBirthMappings.map(([k, v]) => ({ 'V1 Field': k, 'V2 Field': v })))

console.log('Death:')
render(badDeathMappings.map(([k, v]) => ({ 'V1 Field': k, 'V2 Field': v })))

console.log()
console.log('V1 form fields mapped but V2 field does not have a resolver:')
console.log('Birth:')
render(
  unResolvedBirthFields.map(([k, v]) => ({ 'V1 Field': k, 'V2 Field': v }))
)
console.log('Death:')
render(
  unResolvedDeathFields.map(([k, v]) => ({ 'V1 Field': k, 'V2 Field': v }))
)

console.log()
console.log('Additional V2 form fields not mapped:')
console.log('Birth:')
render(unmappedFields.birth.extraV2Mappings)

console.log('Death:')
render(unmappedFields.death.extraV2Mappings)

if (
  unmappedFields.birth.missingV1Mappings.length ||
  unmappedFields.death.missingV1Mappings.length ||
  badBirthMappings.length ||
  badDeathMappings.length ||
  unResolvedBirthFields.length ||
  unResolvedDeathFields.length
) {
  throw new Error('Migration may not proceed. Please fix errors')
} else {
  console.log()
  console.log('Ok to proceed with migration')
}
